<a href="https://colab.research.google.com/github/Ananthrkulks/Ananthrkulks/blob/main/Malaria_Detection_by_Neuralearn_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,GlobalAveragePooling2D,Activation,Dropout, BatchNormalization,InputLayer # Fixed import
from tensorflow.python.keras import Sequential,backend,optimizers
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

**DATA PREPARATION**

In [ ]:
dataset, dataset_info = tfds.load('malaria', with_info=True, as_supervised=True, shuffle_files=True, split=['train'])




In [ ]:
dataset



In [ ]:
dataset_info


**DATASET VISUALIZATION**

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt


dataset, dataset_info = tfds.load('malaria', with_info=True, as_supervised=True, shuffle_files=True, split='train')

TRAIN_RATIO = 0.6
VAL_RATIO = 0.2
TEST_RATIO = 0.2


DATASET_SIZE = len(list(dataset))

train_dataset = dataset.take(int(TRAIN_RATIO * DATASET_SIZE))
val_test_dataset = dataset.skip(int(TRAIN_RATIO * DATASET_SIZE))
val_dataset = val_test_dataset.take(int(VAL_RATIO * DATASET_SIZE))
test_dataset = val_test_dataset.skip(int(VAL_RATIO * DATASET_SIZE))

for i, (image, label) in enumerate(train_dataset.take(16)):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(image)
    plt.show()



In [ ]:
dataset_info.features['label'].int2str(0)

**DATA PREPROCESSING**

In [ ]:
IM_SIZE = 224
def resizing(image, label):
  return tf.image.resize(image,(IM_SIZE,IM_SIZE))/255.0 , label

In [ ]:
train_dataset = train_dataset.map(resizing)
val_dataset = val_dataset.map(resizing)
test_dataset = test_dataset.map(resizing)

train_dataset

**MODEL CREATION AND TRAINING**

In [ ]:
IM_SIZE = 224
def resizing(image, label):
  if image.shape.ndims == 2:
    image = tf.expand_dims(image, axis=-1)
  return tf.image.resize(image,(IM_SIZE,IM_SIZE))/255.0 , label

In [ ]:
for image, label in train_dataset.take(1):
  print(image, label)

In [ ]:
BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
BATCH_SIZE = 32
val_dataset = val_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
BATCH_SIZE = 32
test_dataset = test_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
model = tf.keras.Sequential([
                             InputLayer(input_shape = (IM_SIZE, IM_SIZE, 3)),
                             tf.keras.layers.Conv2D(filters = 6, kernel_size = 5, strides = 1, padding='valid',activation='sigmoid') # Corrected the typo here
])
model.summary()

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.InputLayer(input_shape = (IM_SIZE, IM_SIZE, 3)), # Use tf.keras.layers.InputLayer
                             tf.keras.layers.Conv2D(filters = 6, kernel_size = 5, strides = 1, padding='valid', activation='sigmoid') # Corrected kernel_size
])
model.summary()

In [ ]:
lenet_model = tf.keras.Sequential([
    InputLayer(input_shape=(IM_SIZE, IM_SIZE, 3)),
    tf.keras.layers.Conv2D(filters=6, kernel_size=3, strides=1, padding='valid', activation='relu'),
    BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),

    tf.keras.layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu'),
    BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),

    Flatten(),

    Dense(100, activation='relu'),
    BatchNormalization(),

    Dense(10, activation='relu'),
    BatchNormalization(),

    Dense(1, activation='sigmoid'),

])

lenet_model.summary()

In [ ]:
y_true = [0, 1, 0, 0]
y_pred = [0.6, 0.51, 0.94, 1]

y_true = np.array(y_true)
y_pred = np.array(y_pred)

bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
bce(y_true, y_pred)

In [ ]:
lenet_model.compile(optimizer = Adam(learning_rate = 0.01),
              loss = BinaryCrossentropy(),
              metrics = ['accuracy'])

In [ ]:
history = lenet_model.fit(train_dataset,validation_data = val_dataset, epochs = 20, verbose = 1)

**MODEL EVALUATION AND TESTING**

In [ ]:
val_dataset

In [ ]:
train_dataset

In [ ]:
test_dataset

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train_accuracy', 'val_accuracy'])
plt.show()


In [ ]:
lenet_model.evaluate (test_dataset)

In [ ]:
train_dataset

In [ ]:
test_dataset = val_test_dataset.skip(int(VAL_RATIO * DATASET_SIZE))

In [ ]:
test_dataset

In [ ]:
def parasite_or_not(x):
  if(x<0.5):
    return 'Parasitized'
  else:
    return 'Uninfected'

In [ ]:
for i, (image, label) in enumerate(train_dataset.take(9)):

    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image[0])
    plt.title(str(parasite_or_not(label.numpy()[0])) + ":" + str(parasite_or_not(lenet_model.predict(image)[0][0])))

    plt.axis('off')

**LOADING AND SAVING **

In [ ]:
lenet_model.save("tensorflow_env1.keras")

In [ ]:
lenet_model.save("tensorflow_env1.h5")

In [ ]:
lenet_model.save("lenet.h5")

In [ ]:
lenet_model.save("lenet.h5")

lenet_loaded_model = tf.keras.models.load_model("lenet.h5")
lenet_loaded_model.summary()

In [ ]:
import os
os.makedirs("weights", exist_ok=True)
lenet_model.save_weights("weights/lenet_weights.weights.h5")

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!cp -r /content/lenet.h5 /content/drive/MyDrive/